In [ ]:
%env OPENAI_API_KEY=keyhere

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio
from kagent.tools.docs import search_documentation

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

kubernetes_docs_agent = AssistantAgent(
    "kubernetes_docs_agent",
    description="This agent allows you to get data from the Kubernetes docs database.",
    tools=[search_documentation],
    model_client=model_client,
    system_message="""
    You are a support agent.

    You have access to several tools:
    - 'searchDocumentation' to search in the documentation.

    Execute all the following steps:
    1. Product identification
    - Check if you know what product the question is about (Only supported product is: 'kubernetes')
    - If it's not the case, ask what the product is

    2. Information Gathering
    - Search in the documentation for information related to the question the user has submitted

    3. Answer the question
    - Use all the information you gathered to provide a valuable answer to the user
    - Provide links to the documentation whenever possible`,
    """,
)

user_proxy = UserProxyAgent("user_proxy", input_func=input)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    [
        kubernetes_docs_agent,
    ],
    termination_condition=termination,
)

task = "How do I do a rolling update of a deployment in kubernetes?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
How do I do a rolling update of a deployment in kubernetes?
---------- kubernetes_docs_agent ----------
[FunctionCall(id='call_9V1ziNjJ5RD5tXCVhqNSfpHc', arguments='{"query":"rolling update of a deployment","product":"kubernetes","version":null,"limit":5}', name='search_documentation')]
---------- kubernetes_docs_agent ----------
[FunctionExecutionResult(content='[{"distance": 0.8394350409507751, "content": "\\nSimilar to application Scaling, if a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update. An available Pod is an instance that is available to the users of the application.\\n\\nRolling updates allow the following actions:\\n\\n*   Promote an application from one environment to another (via container image updates)\\n*   Rollback to previous versions\\n*   Continuous Integration and Continuous Delivery of applications with zero downtime\\n\\n_If a Deployment is exposed publicly, the Service

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='How do I do a rolling update of a deployment in kubernetes?', type='TextMessage'), ToolCallRequestEvent(source='kubernetes_docs_agent', models_usage=RequestUsage(prompt_tokens=441, completion_tokens=33), content=[FunctionCall(id='call_9V1ziNjJ5RD5tXCVhqNSfpHc', arguments='{"query":"rolling update of a deployment","product":"kubernetes","version":null,"limit":5}', name='search_documentation')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='kubernetes_docs_agent', models_usage=None, content=[FunctionExecutionResult(content='[{"distance": 0.8394350409507751, "content": "\\nSimilar to application Scaling, if a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update. An available Pod is an instance that is available to the users of the application.\\n\\nRolling updates allow the following actions:\\n\\n*   Promote an application from one envir

In [ ]:
print(team.dump_component().model_dump_json(indent=2))